# Práctica 2.A Toma de contacto con AIMA

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se implementan algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [1]:
from search import *

In [2]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [3]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [4]:
p =Jarras()
p.initial

(0, 0)

In [5]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [6]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [7]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [8]:
p.goal_test(p.initial)

False

# Representa el problema del puzzle de 8

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. Tenlo en cuenta al hacer las pruebas. 

### EJERCICIO 1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 

In [9]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")    
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
            
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0  
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)
    
    def h(self, node):
        " Return the heuristic value for a given state."
        return 1

### Ejercicio 1.3. Probar los siguientes ejemplos que se pueden ejecutar una vez se ha definido la clase:

In [10]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [11]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [12]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [13]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [14]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [15]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras como al problema del ocho puzzle con distintos estados iniciales.

In [16]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [17]:
breadth_first_tree_search(Jarras()).solution()


['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [18]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el puzzle de 8

In [19]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [20]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [21]:
#breadth_first_tree_search(EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# Busqueda en anchura sin control de repetidos 

In [22]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [23]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [24]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [23]:
depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

In [ ]:
breadth_first_graph_search(estado).solution()

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

--------------------------------
--------------------------------------------------------------------------------------------------
 --------------------------
 -----------------------------------------------------
 -------------------------
La búsqueda en profundidad tarda demaisado. La conclusión es que el algoritmo se ha metido en un bucle y, no reconociéndolo, no encuentra una solución. 
La búsqueda en anchura, por otro lado, es una búsqueda completa y encuentra una solución en alguno de los nodos ya que al menos uno de los desarrollos no entrará en un bucle.
**********************************************************************************



### Ejercicio 3: definir al menos las siguientes funciones heurísticas para el 8 puzzle:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [25]:
# Heuristicas para el 8 Puzzle 

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    count = 0
    for i in goal:
        if node.state[i] != goal[i]: 
            count = count+1 
        
    return count

def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    stat = 0
    gol = 0
    mhd = 0
    for i in state:
        if state[i] != goal[i]:
            stat = state.index(state[i])
            gol = goal.index(state[i])
            #asignamos una tupla a la posición de el elemento i en la lista del estado actual
            pos_state = (int(stat / 3), int(stat % 3))
            #asignamos una tupla a la posición de el elemento i en la lista del estado final
            pos_goal = (int(gol / 3), int(gol % 3))
            #añadimos la distancia manhattan del elemento dado hasta su posición final
            a1 = abs(pos_goal[0]-pos_state[0])
            a2 = abs(pos_goal[1]-pos_state[1])
            mhd = mhd + a1 + a2
    
    return mhd

def sqrt_manhattan(node):
    state = node.state
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente estado inicial y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [26]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [27]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [28]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [29]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
astar_search(puzle,linear).solution()

In [30]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [31]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [32]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

3.04 ms ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

2.59 ms ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

40.8 ms ± 4.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

2.8 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

# ----------------------------------------------------------------------------
La mejor heurística es la Manhattan ya que no solo tarda menos si no que, debido al 'if', no comprueba los elementos que ya están bien colocados.
# ----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion.
En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando el número de inversiones. Si es impar el tablero no tiene solución. 

The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.

In [36]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        

In [37]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)
h2_ocho_puzzle = manhattan

In [38]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [39]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [40]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, h2_ocho_puzzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [41]:
astar_search(p8, h2_ocho_puzzle).solution()

['Mover hueco derecha']

In [42]:
astar_search(p8p, h2_ocho_puzzle).solution()

['Mover hueco derecha']

In [43]:
def resuelve_ocho_puzzle(estado_init, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_init))
    if p8p.check_solvability(estado_init):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [44]:
resuelve_ocho_puzzle(estado_inicial,astar_search,h2_ocho_puzzle)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


#### Ejercicio 5

#Intentar resolver usando las distintas búsquedas y en su caso, las distintas
#heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 8 | 3 |   | 4 | 8 | 1 |   | 2 | 1 | 6 |   | 5 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 1 | 6 | 4 |   | 3 | H | 2 |   | 4 | H | 8 |   | H | 4 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 6 | 5 |   | 7 | 5 | 3 |   | 7 | 6 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 justificarlos con las distintas propiedades teóricas estudiadas.

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
    Anchura                            L=17         L=11         L= ∞        L=3
                                       T=30.46min   T=1.21s      T= ∞        T=0ns  
                                       NA=31437904  NA=134082    NA= ∞       NA=24 
   
    Profundidad                        L=7977       L=52429      L=20        L=27  
                                       T=10.1s      T=8.28min    T=4.39min   T=0ns 
                                       NA=8198      NA=60036     NA=48428    NA=28 
                                       
    Coste Uniforme                     L=17         L=11         L=20        L=3
                                       T=53.3s      T=219ms      T=7.32min   T=0ns 
                                       NA=14092     NA=870       NA=48428    NA=13 
                                       
    Primero el mejor (linear)          L=39         L=33         L=76        L=3
                                       T=13.6ms     T=4.39ms     T=748       T=0ns  
                                       NA=173       NA=78        NA=131ms    NA=4
                                                                            
    Primero el mejor (manhattan)       L=67         L=15         L=74        L=3 
                                       T=135ms      T=2.95ms     T=89.7ms    T=0ns  
                                       NA=651       NA=26        NA=591      NA=4
                                                                             
    A* (linear)                        L=17         L=11         L=20        L=3 
                                       T=206ms      T=6.3ms      T=2.29s     T=0ns  
                                       NA=873       NA=77        NA=3377     NA=4                                                                           
     A* (manhattan)                    L=17         L=11         L=20        L=3 
                                       T=56ms       T=2.25       T=316ms     T=0ns  
                                       NA=362       NA=39        NA=1153     NA=4

 -----------------------------------------------------------------------------------------




In [45]:
puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
resuelve_ocho_puzzle(puzzle_1,breadth_first_tree_search)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 8. Nodos analizados: 2496


In [46]:
estado_inicial1 = (2, 1, 3, 4, 8, 6, 7, 0, 5)
estado_inicial2 = (1, 0, 3, 4, 8, 6, 7, 2, 5)
estado_inicial3 = (4, 5, 6, 1, 0, 3, 7, 8, 2)
estado_inicial4 = (1, 2, 3, 0, 5, 6, 4, 7, 8)

In [47]:
#PUZLE 1 = (2, 1, 3, 4, 8, 6, 7, 0, 5)

In [48]:
%%time
resuelve_ocho_puzzle(estado_inicial1,breadth_first_tree_search)

KeyboardInterrupt: 

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,uniform_cost_search)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,best_first_graph_search,linear)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,best_first_graph_search,h2_ocho_puzzle)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,astar_search,linear)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial1,astar_search,h2_ocho_puzzle)

In [ ]:
#PUZLE 2 = (1, 0, 3, 4, 8, 6, 7, 2, 5)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,breadth_first_tree_search)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,depth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,uniform_cost_search)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,best_first_graph_search,linear)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,best_first_graph_search,h2_ocho_puzzle)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,astar_search,linear)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial2,astar_search,h2_ocho_puzzle)

In [ ]:
#PUZLE 3 = (4, 5, 6, 1, 0, 3, 7, 8, 2)

In [ ]:
##########################################################################################
##########################################################################################
#########################################################################################
##########################################################################################
%%time
resuelve_ocho_puzzle(estado_inicial3,breadth_first_tree_search)
##########################################################################################
##########################################################################################
###########################   NO   TERMINA   ############################################
##########################################################################################

In [115]:
%%time
resuelve_ocho_puzzle(estado_inicial3,depth_first_graph_search)

KeyboardInterrupt: 

In [116]:
%%time
resuelve_ocho_puzzle(estado_inicial3,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428
Wall time: 7min 32s


In [117]:
%%time
resuelve_ocho_puzzle(estado_inicial3,best_first_graph_search,linear)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [118]:
%%time
resuelve_ocho_puzzle(estado_inicial3,best_first_graph_search,h2_ocho_puzzle)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco 

In [119]:
%%time
resuelve_ocho_puzzle(estado_inicial3,astar_search,linear)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3377
Wall time: 2.29 s


In [120]:
%%time
resuelve_ocho_puzzle(estado_inicial3,astar_search,h2_ocho_puzzle)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 1153
Wall time: 316 ms


In [ ]:
#PUZLE 4 = (1, 2, 3, 0, 5, 6, 4, 7, 8)

In [130]:
%%time
resuelve_ocho_puzzle(estado_inicial4,breadth_first_tree_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_tree_search
Longitud de la solución: 3. Nodos analizados: 24
Wall time: 0 ns


In [129]:
%%time
resuelve_ocho_puzzle(estado_inicial4,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: depth_first_graph_search
Longitud de la solución: 27. Nodos analizados: 28
Wall time: 0 ns


In [128]:
%%time
resuelve_ocho_puzzle(estado_inicial4,uniform_cost_search)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 3. Nodos analizados: 13
Wall time: 0 ns


In [127]:
%%time
resuelve_ocho_puzzle(estado_inicial4,best_first_graph_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
Wall time: 0 ns


In [126]:
%%time
resuelve_ocho_puzzle(estado_inicial4,best_first_graph_search,h2_ocho_puzzle)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
Wall time: 0 ns


In [125]:
%%time
resuelve_ocho_puzzle(estado_inicial4,astar_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
Wall time: 0 ns


In [124]:
%%time
resuelve_ocho_puzzle(estado_inicial4,astar_search,h2_ocho_puzzle)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
Wall time: 997 µs


### Ejercicio 6: [OPCIONAL] Definir una heurística más informada y completa la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [456]:
# Proponemmos como heurística h3 la se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.
# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

         

# ---------------------------------------------------------------------------
# sumaSecuencia = si la ficha es distinta de 0 :
#          - si está en el centro => 1
#          - si no esta en el centro y su sucesor en la secuencia no es el correspondiente => 2
# heuristica = manhattan + 3 * sumaSecuencias
